In [120]:
import os
from bs4 import BeautifulSoup
from itertools import groupby
from pathlib import Path

from jedi.inference import follow_error_node_imports_if_possible


In [ ]:
doc_dirs = []
for n in range(1,6):
    dirpath = "/srv/data/greek/OGA/oga/oga_v0.2.0_{}/".format(str(n))
    docdir_names = os.listdir(dirpath.format(str(n)))
    doc_dirs.extend([dirpath + docdir_name for docdir_name in docdir_names])
len(doc_dirs)

In [ ]:

doc_dirs[1000:1050]

In [ ]:
doc_dir = "tlg0014.tlg005.perseus-grc2"

In [131]:
os.listdir(doc_dirs[0])

['anno_urn_cts_edition.xml',
 'dep.tlg0007.tlg082.perseus-grc3.tok01_dep01.xml',
 'anno_urn_cts.xml',
 'anno_urn_cts_work.xml',
 'anno_urn_cts_author.xml',
 'anno_title.xml',
 'tlg0007.tlg082.perseus-grc3.tok01_ipa01.xml',
 'postag.tlg0007.tlg082.perseus-grc3.tok01_postag01.xml',
 'tlg0007.tlg082.perseus-grc3.xml',
 'tlg0007.tlg082.perseus-grc3.tok01_lemma01.xml',
 'anno.xml',
 'tlg0007.tlg082.perseus-grc3.tok01_cts01.xml',
 'anno_author.xml',
 'anno_work_date.xml',
 'tlg0007.tlg082.perseus-grc3.tok01.xml',
 'dep.tlg0007.tlg082.perseus-grc3.tok01_dep01_dep-fnc01.xml']

In [151]:
oga_cts_author, oga_cts_work, oga_cts_source = doc_dirs[1998].rpartition("/")[-1].split(".")
print(oga_cts_author, oga_cts_work, oga_cts_source)

tlg2200 tlg00444 opp-grc1


In [139]:

file_path = doc_dirs[0] + "/anno_urn_cts_work.xml"
# Open the XML file
with open(file_path, 'r', encoding='utf-8') as file:
    # Parse the file using BeautifulSoup
    soup_fealist = BeautifulSoup(file, 'xml')  # Use 'xml' parser for XML data

# Print the root tag to confirm loading
print(soup_fealist.prettify())

<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE paula SYSTEM "../../paula_dtd/paula_feat.dtd">
<paula version="1.1">
 <header paula_id="anno_urn_cts_work"/>
 <featList type="urn_cts_work" xml:base="anno.xml" xmlns:xlink="http://www.w3.org/1999/xlink">
  <feat value="tlg082" xlink:href="#anno"/>
 </featList>
</paula>



In [ ]:

file_path = doc_dirs[0] + "/tlg0007.tlg082.perseus-grc3.tok01_cts01.xml"
# Open the XML file
with open(file_path, 'r', encoding='utf-8') as file:
    # Parse the file using BeautifulSoup
    soup_fealist = BeautifulSoup(file, 'xml')  # Use 'xml' parser for XML data

# Print the root tag to confirm loading
print(soup_fealist.prettify())

In [ ]:

file_path = doc_dirs[0] + "/tlg0007.tlg082.perseus-grc3.tok01.xml"
# Open the XML file
with open(file_path, 'r', encoding='utf-8') as file:
    # Parse the file using BeautifulSoup
    soup_tokens = BeautifulSoup(file, 'xml')  # Use 'xml' parser for XML data

# Print the root tag to confirm loading
print(soup_tokens.prettify()[:10000])

In [ ]:
file_path = doc_dirs[0] + "/tlg0007.tlg082.perseus-grc3.tok01_lemma01.xml"
# Open the XML file
with open(file_path, 'r', encoding='utf-8') as file:
    # Parse the file using BeautifulSoup
    soup_tokens = BeautifulSoup(file, 'xml')  # Use 'xml' parser for XML data

# Print the root tag to confirm loading
print(soup_tokens.prettify()[:10000])

In [ ]:
file_path = doc_dirs[0] + "/postag.tlg0007.tlg082.perseus-grc3.tok01_postag01.xml"
# Open the XML file
with open(file_path, 'r', encoding='utf-8') as file:
    # Parse the file using BeautifulSoup
    soup_postags = BeautifulSoup(file, 'xml')  # Use 'xml' parser for XML data

# Print the root tag to confirm loading
print(soup_postags.prettify()[:10000])

In [127]:
# Define file suffixes we're interested in
FILE_SUFFIXES = {
    "tokens": ".tok01.xml",
    "lemmas": ".tok01_lemma01.xml",
    "pos": ".tok01_postag01.xml",
    "cts": ".tok01_cts01.xml",
}


# Function to gather file paths from directories
def gather_file_paths(doc_dirs):
    file_structure = {}

    for subfolder in doc_dirs:
        subfolder_name = Path(subfolder).name  # Get the folder name
        file_structure[subfolder_name] = {}  # Create a key for the subfolder

        for file_type, suffix in FILE_SUFFIXES.items():
            # Search for files matching each suffix in the current subfolder
            matching_files = list(Path(subfolder).glob(f"*{suffix}"))

            if matching_files:
                # Use the first matching file (assuming one per type per subfolder)
                file_structure[subfolder_name][file_type] = str(matching_files[0])
            else:
                # No file found for this type; mark as None
                file_structure[subfolder_name][file_type] = None

    return file_structure

In [130]:
doc_dirs[:10]

['/srv/data/greek/OGA/oga/oga_v0.2.0_1/tlg0007.tlg082.perseus-grc3',
 '/srv/data/greek/OGA/oga/oga_v0.2.0_1/pta0003.pta009.pta-grc1',
 '/srv/data/greek/OGA/oga/oga_v0.2.0_1/tlg0007.tlg134.perseus-grc2',
 '/srv/data/greek/OGA/oga/oga_v0.2.0_1/pta9999.pta016.pta-grc1',
 '/srv/data/greek/OGA/oga/oga_v0.2.0_1/tlg0010.tlg007.perseus-grc2',
 '/srv/data/greek/OGA/oga/oga_v0.2.0_1/stoa0033a.tlg043.1st1K-grc1',
 '/srv/data/greek/OGA/oga/oga_v0.2.0_1/pta9999.pta055.pta-grc1',
 '/srv/data/greek/OGA/oga/oga_v0.2.0_1/tlg0007.tlg045.perseus-grc2',
 '/srv/data/greek/OGA/oga/oga_v0.2.0_1/pta9999.pta075.pta-grc1',
 '/srv/data/greek/OGA/oga/oga_v0.2.0_1/pta0001.pta054.pta-grc1']

In [128]:
file_paths = gather_file_paths(doc_dirs)

In [129]:
len(file_paths)

1999

In [ ]:


# Utility to extract positional indices from xlink:href in tokens
def extract_position(xlink):
    pos = xlink.split("string-range(//body,'',")[-1].strip("))").split(",")
    return tuple(map(int, pos))  # Converts "1,9" to (1, 9)


# Parse all files in parallel
with open(files["tokens"], "r", encoding="utf-8") as f_tokens, \
        open(files["lemmas"], "r", encoding="utf-8") as f_lemmas, \
        open(files["pos"], "r", encoding="utf-8") as f_pos, \
        open(files["cts"], "r", encoding="utf-8") as f_cts:
    # Parse XML files
    soup_tokens = BeautifulSoup(f_tokens, "xml")
    soup_lemmas = BeautifulSoup(f_lemmas, "xml")
    soup_pos = BeautifulSoup(f_pos, "xml")
    soup_cts = BeautifulSoup(f_cts, "xml")

    # Extract data
    # Tokens
    tokens_data = {}
    for mark in soup_tokens.find_all("mark"):
        token_id = mark["id"]

        # Extract token from adjacent XML comment
        token_comment = mark.find_next(string=lambda text: isinstance(text, Comment))
        token_string = token_comment.strip() if token_comment else ""  # Handle missing comments gracefully

        tokens_data[token_id] = {
            "token": token_string,
            "pos_index": extract_position(mark["xlink:href"])
        }

    # Lemmas
    for feat in soup_lemmas.find_all("feat"):
        token_id = feat["xlink:href"].lstrip("#")
        if token_id in tokens_data:
            tokens_data[token_id]["lemma"] = feat["value"]

    # POS tags
    for multiFeat in soup_pos.find_all("multiFeat"):
        token_id = multiFeat["xlink:href"].lstrip("#")
        if token_id in tokens_data:
            tokens_data[token_id]["pos"] = multiFeat.find("feat", {"name": "pos"})["value"]

    # Subunits/Sentence IDs
    for feat in soup_cts.find_all("feat"):
        token_id = feat["xlink:href"].lstrip("#")
        if token_id in tokens_data:
            tokens_data[token_id]["subunit"] = feat["value"]

# Combine into list of tuples
token_tuples = [
    (
        data.get("token", ""),
        data.get("lemma", ""),
        data.get("pos", ""),
        data.get("subunit", ""),
        data.get("pos_index", ())
    )
    for data in tokens_data.values()
]

# Output as requested
print(token_tuples)

In [109]:
token_tuples[:20]

[('Ἀγασικλῆς', 'ἀγασικλῆς', 'n', '1_1', (1, 9)),
 ('ὁ', 'ὁ', 'l', '1_1', (11, 1)),
 ('Λακεδαιμονίων', 'λακεδαιμόνιος', 'n', '1_1', (13, 13)),
 ('βασιλεύς', 'βασιλεύς', 'n', '1_1', (27, 8)),
 (',', ',', 'u', '1_1', (36, 1)),
 ('θαυμάζοντός', 'θαυμάζω', 'v', '1_1', (38, 11)),
 ('τινος', 'τις', 'p', '1_1', (50, 5)),
 ('ὅτι', 'ὅτι', 'c', '1_1', (56, 3)),
 ('φιλήκοος', 'φιλήκοος', 'a', '1_1', (60, 8)),
 ('ὢν', 'εἰμί', 'v', '1_1', (69, 2)),
 ('οὐ', 'οὐ', 'd', '1_1', (72, 2)),
 ('προσδέχεται', 'προσδέχομαι', 'v', '1_1', (75, 11)),
 ('Φιλοφάνη', 'φιλοφάνης', 'n', '1_1', (87, 8)),
 ('τὸν', 'ὁ', 'l', '1_1', (96, 3)),
 ('σοφιστήν', 'σοφιστής', 'n', '1_1', (100, 8)),
 (',', ',', 'u', '1_1', (109, 1)),
 ('ἔφη', 'φημί', 'v', '1_1', (111, 3)),
 (',', ',', 'u', '1_1', (115, 1)),
 ('τούτων', 'οὗτος', 'p', '1_1', (117, 6)),
 ('χρῄζω', 'χρῄζω', 'v', '1_1', (124, 5))]

In [118]:
# Process sentences
sentences_data = []

for key, group in groupby(token_tuples, key=lambda x: x[3]):  # Group by subunit
    group_list = list(group)
    # Find the maximum index to determine sentence length
    max_end = max(start + length - 1 for _, _, _, _, (start, length) in group_list)

    # Create a blank sentence field (filled with spaces)
    sentence_field = [' '] * max_end

    # Insert each token into the sentence field based on its position
    recalculated_group = []
    for surface, lemma, pos, subunit, (start, length) in group_list:
        adjusted_start = start - 1  # Convert human index to Python index (0-based)
        adjusted_end = adjusted_start + length
        sentence_field[adjusted_start:adjusted_end] = surface  # Place the surface form in positions

        # Store recalculated token data
        recalculated_group.append((surface, lemma, pos, (adjusted_start, adjusted_end)))

    # Join the sentence field into the raw sentence text
    raw_sentence = ''.join(sentence_field)

    # Collect the sentence data
    sentences_data.append((key, raw_sentence, recalculated_group))


In [119]:
sentences_data[:3]

[('1_1',
  'Ἀγασικλῆς ὁ Λακεδαιμονίων βασιλεύς , θαυμάζοντός τινος ὅτι φιλήκοος ὢν οὐ προσδέχεται Φιλοφάνη τὸν σοφιστήν , ἔφη , τούτων χρῄζω μαθητὴς εἶναι , ὧν εἶναι καὶ υἱός .',
  [('Ἀγασικλῆς', 'ἀγασικλῆς', 'n', (0, 9)),
   ('ὁ', 'ὁ', 'l', (10, 11)),
   ('Λακεδαιμονίων', 'λακεδαιμόνιος', 'n', (12, 25)),
   ('βασιλεύς', 'βασιλεύς', 'n', (26, 34)),
   (',', ',', 'u', (35, 36)),
   ('θαυμάζοντός', 'θαυμάζω', 'v', (37, 48)),
   ('τινος', 'τις', 'p', (49, 54)),
   ('ὅτι', 'ὅτι', 'c', (55, 58)),
   ('φιλήκοος', 'φιλήκοος', 'a', (59, 67)),
   ('ὢν', 'εἰμί', 'v', (68, 70)),
   ('οὐ', 'οὐ', 'd', (71, 73)),
   ('προσδέχεται', 'προσδέχομαι', 'v', (74, 85)),
   ('Φιλοφάνη', 'φιλοφάνης', 'n', (86, 94)),
   ('τὸν', 'ὁ', 'l', (95, 98)),
   ('σοφιστήν', 'σοφιστής', 'n', (99, 107)),
   (',', ',', 'u', (108, 109)),
   ('ἔφη', 'φημί', 'v', (110, 113)),
   (',', ',', 'u', (114, 115)),
   ('τούτων', 'οὗτος', 'p', (116, 122)),
   ('χρῄζω', 'χρῄζω', 'v', (123, 128)),
   ('μαθητὴς', 'μαθητής', 'n', (129, 13

In [171]:
def paths_to_soups(doc_dir):
    path_tokens = str(list(Path(doc_dir).glob(f"*{".tok01.xml"}"))[0])
    path_lemmas = str(list(Path(doc_dir).glob(f"*{".tok01_lemma01.xml"}"))[0])
    path_pos = str(list(Path(doc_dir).glob(f"*{".tok01_postag01.xml"}"))[0])
    path_cts = str(list(Path(doc_dir).glob(f"*{".tok01_cts01.xml"}"))[0])
    with open(path_tokens, "r", encoding="utf-8") as f_tokens, \
            open(path_lemmas, "r", encoding="utf-8") as f_lemmas, \
            open(path_pos, "r", encoding="utf-8") as f_pos, \
            open(path_cts, "r", encoding="utf-8") as f_cts:
    # Parse XML files
        soup_tokens = BeautifulSoup(f_tokens, "xml")
        soup_lemmas = BeautifulSoup(f_lemmas, "xml")
        soup_pos = BeautifulSoup(f_pos, "xml")
        soup_cts = BeautifulSoup(f_cts, "xml")
    return soup_tokens, soup_lemmas, soup_pos, soup_cts

def extract_token_data(soup_tokens, soup_lemmas, soup_pos, soup_cts):
        # Extract data
    # Tokens
    tokens_data = {}
    for mark in soup_tokens.find_all("mark"):
        token_id = mark["id"]

        # Extract token from adjacent XML comment
        token_comment = mark.find_next(string=lambda text: isinstance(text, Comment))
        token_string = token_comment.strip() if token_comment else ""  # Handle missing comments gracefully

        tokens_data[token_id] = {
            "token": token_string,
            "pos_index": extract_position(mark["xlink:href"])
        }

    # Lemmas
    for feat in soup_lemmas.find_all("feat"):
        token_id = feat["xlink:href"].lstrip("#")
        if token_id in tokens_data:
            tokens_data[token_id]["lemma"] = feat["value"]

    # POS tags
    for multiFeat in soup_pos.find_all("multiFeat"):
        token_id = multiFeat["xlink:href"].lstrip("#")
        if token_id in tokens_data:
            tokens_data[token_id]["pos"] = multiFeat.find("feat", {"name": "pos"})["value"]

    # Subunits/Sentence IDs
    for feat in soup_cts.find_all("feat"):
        token_id = feat["xlink:href"].lstrip("#")
        if token_id in tokens_data:
            tokens_data[token_id]["subunit"] = feat["value"]

    # Combine into list of tuples
    token_tuples = [
        (
            data.get("token", ""),
            data.get("lemma", ""),
            data.get("pos", ""),
            data.get("subunit", ""),
            data.get("pos_index", ())
        )
        for data in tokens_data.values()
    ]
    return token_tuples

def extract_sentence_data(token_tuples):
    # Process sentences
    sentences_data = []

    for key, group in groupby(token_tuples, key=lambda x: x[3]):  # Group by subunit
        group_list = list(group)

        # Determine the starting point of the sentence to reset indexing
        min_start = min(start for _, _, _, _, (start, length) in group_list)

        # Calculate relative positions for this sentence
        recalculated_group = []
        sentence_length = 0  # Track total length needed for the sentence's surface

        for surface, lemma, pos, subunit, (start, length) in group_list:
            # Adjust the position to start from 0 for the sentence
            adjusted_start = start - min_start  # Shift start relative to min_start (relative indexing starts at 0)
            adjusted_end = adjusted_start + length

            # Append the token with recalculated positions
            recalculated_group.append((surface, lemma, pos, (adjusted_start, adjusted_end)))

            # Track the sentence length dynamically
            sentence_length = max(sentence_length, adjusted_end)

        # Create a blank sentence field (filled with spaces to start with)
        sentence_field = [' '] * sentence_length

        # Place the tokens into the sentence field at their adjusted positions
        for surface, _, _, (adjusted_start, adjusted_end) in recalculated_group:
            sentence_field[adjusted_start:adjusted_end] = surface  # Insert the surface at the specified range

        # Join the sentence field into the raw sentence text
        raw_sentence = ''.join(sentence_field)

        # Collect the sentence data: subunit key, raw text, and recalculated tokens
        sentences_data.append((key, raw_sentence, recalculated_group))

    return sentences_data


In [172]:
soup_tokens, soup_lemmas, soup_pos, soup_cts = paths_to_soups(doc_dirs[0])
token_tuples = extract_token_data(soup_tokens, soup_lemmas, soup_pos, soup_cts)
sentences_data = extract_sentence_data(token_tuples)
sentences_data[:3]

[('1_1',
  'Ἀγασικλῆς ὁ Λακεδαιμονίων βασιλεύς , θαυμάζοντός τινος ὅτι φιλήκοος ὢν οὐ προσδέχεται Φιλοφάνη τὸν σοφιστήν , ἔφη , τούτων χρῄζω μαθητὴς εἶναι , ὧν εἶναι καὶ υἱός .',
  [('Ἀγασικλῆς', 'ἀγασικλῆς', 'n', (0, 9)),
   ('ὁ', 'ὁ', 'l', (10, 11)),
   ('Λακεδαιμονίων', 'λακεδαιμόνιος', 'n', (12, 25)),
   ('βασιλεύς', 'βασιλεύς', 'n', (26, 34)),
   (',', ',', 'u', (35, 36)),
   ('θαυμάζοντός', 'θαυμάζω', 'v', (37, 48)),
   ('τινος', 'τις', 'p', (49, 54)),
   ('ὅτι', 'ὅτι', 'c', (55, 58)),
   ('φιλήκοος', 'φιλήκοος', 'a', (59, 67)),
   ('ὢν', 'εἰμί', 'v', (68, 70)),
   ('οὐ', 'οὐ', 'd', (71, 73)),
   ('προσδέχεται', 'προσδέχομαι', 'v', (74, 85)),
   ('Φιλοφάνη', 'φιλοφάνης', 'n', (86, 94)),
   ('τὸν', 'ὁ', 'l', (95, 98)),
   ('σοφιστήν', 'σοφιστής', 'n', (99, 107)),
   (',', ',', 'u', (108, 109)),
   ('ἔφη', 'φημί', 'v', (110, 113)),
   (',', ',', 'u', (114, 115)),
   ('τούτων', 'οὗτος', 'p', (116, 122)),
   ('χρῄζω', 'χρῄζω', 'v', (123, 128)),
   ('μαθητὴς', 'μαθητής', 'n', (129, 13

In [162]:
soup_cts

<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE paula SYSTEM "../../paula_dtd/paula_feat.dtd">
<paula version="1.1">
<header paula_id="tlg0640.tlg001.1st1K-grc1.tok01_cts01"/>
<featList type="cts" xml:base="tlg0640.tlg001.1st1K-grc1.tok01.xml" xmlns:xlink="http://www.w3.org/1999/xlink">
<feat id="u_1" value="1_1" xlink:href="#t_1"/>
<feat id="u_2" value="1_1" xlink:href="#t_2"/>
<feat id="u_3" value="1_1" xlink:href="#t_3"/>
<feat id="u_4" value="1_1" xlink:href="#t_4"/>
<feat id="u_5" value="1_1" xlink:href="#t_5"/>
<feat id="u_6" value="1_1" xlink:href="#e_1"/>
<feat id="u_7" value="1_1" xlink:href="#t_6"/>
<feat id="u_8" value="1_1" xlink:href="#t_7"/>
<feat id="u_9" value="1_1" xlink:href="#t_8"/>
<feat id="u_10" value="1_1" xlink:href="#e_2"/>
<feat id="u_11" value="1_1" xlink:href="#t_9"/>
<feat id="u_12" value="1_1" xlink:href="#t_10"/>
<feat id="u_13" value="1_1" xlink:href="#t_11"/>
<feat id="u_14" value="1_1" xlink:href="#t_12"/>
<feat id="u_15" value="1_1" xlink:href="#t_13